<a href="https://colab.research.google.com/github/kimkuns91/koGPT/blob/main/koGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
from transformers import AutoTokenizer, AutoModelWithLMHead

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 56.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 61.6 MB/s eta 0:00:00


In [2]:
import torch

In [3]:
model_name = 'skt/kogpt2-base-v2'
Q_TKN = "<usr>"
A_TKN = "<sys>"
BOS = '</s>'
EOS = '</s>'
UNK = '<unk>'
MASK = '<unused0>'
SENT = '<unused1>'
PAD = '<pad>'

In [4]:
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel

In [5]:
koGPT2_Tokenizer = PreTrainedTokenizerFast.from_pretrained(model_name, bos_token=BOS, eos_token=EOS, unk_token = UNK, pad_token = PAD, mask_token = MASK)
model = GPT2LMHeadModel.from_pretrained(model_name)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [6]:
import pandas as pd
chatbot_data = pd.read_csv('/content/drive/MyDrive/DeepLearning/ChatBotData.csv')
chatbot_data.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


In [7]:
chatbot_data['Q']

0                         12시 땡!
1                    1지망 학교 떨어졌어
2                   3박4일 놀러가고 싶다
3                3박4일 정도 놀러가고 싶다
4                        PPL 심하네
                  ...           
11818             훔쳐보는 것도 눈치 보임.
11819             훔쳐보는 것도 눈치 보임.
11820                흑기사 해주는 짝남.
11821    힘든 연애 좋은 연애라는게 무슨 차이일까?
11822                 힘들어서 결혼할까봐
Name: Q, Length: 11823, dtype: object

In [8]:
from torch.utils.data import DataLoader, Dataset

In [9]:
# pytoch 필수 Setting

# class ChabotDataset(Dataset):
#   def __init__():
#   def __len__():
#   def __getitem():

In [10]:
import re
import numpy as np

In [11]:
class ChatbotDataset(Dataset):
    def __init__(self, chats, max_len=10):
        self._data = chats
        self.max_len = max_len
        self.q_token = Q_TKN
        self.a_token = A_TKN
        self.sent_token = SENT
        self.eos = EOS
        self.mask = MASK
        self.tokenizer = koGPT2_Tokenizer
    def __len__(self):
        return len(self._data)
    def __getitem__(self, idx):
        turn = self._data.iloc[idx]
        q = turn["Q"]  # 질문을 가져온다.
        q = re.sub(r"([?.!,])", r" ", q)  # 구둣점들을 제거한다.

        a = turn["A"]  # 답변을 가져온다.
        a = re.sub(r"([?.!,])", r" ", a)  # 구둣점들을 제거한다.

        q_toked = self.tokenizer.tokenize(self.q_token + q + self.sent_token)
        q_len = len(q_toked)

        a_toked = self.tokenizer.tokenize(self.a_token + a + self.eos)
        a_len = len(a_toked)

        #질문의 길이가 최대길이보다 크면
        if q_len > self.max_len:
            a_len = self.max_len - q_len        #답변의 길이를 최대길이 - 질문길이
            if a_len <= 0:       #질문의 길이가 너무 길어 질문만으로 최대 길이를 초과 한다면
                q_toked = q_toked[-(int(self.max_len / 2)) :]   #질문길이를 최대길이의 반으로
                q_len = len(q_toked)
                a_len = self.max_len - q_len              #답변의 길이를 최대길이 - 질문길이
            a_toked = a_toked[:a_len]
            a_len = len(a_toked)

        #질문의 길이 + 답변의 길이가 최대길이보다 크면
        if q_len + a_len > self.max_len:
            a_len = self.max_len - q_len        #답변의 길이를 최대길이 - 질문길이
            if a_len <= 0:       #질문의 길이가 너무 길어 질문만으로 최대 길이를 초과 한다면
                q_toked = q_toked[-(int(self.max_len / 2)) :]   #질문길이를 최대길이의 반으로
                q_len = len(q_toked)
                a_len = self.max_len - q_len              #답변의 길이를 최대길이 - 질문길이
            a_toked = a_toked[:a_len]
            a_len = len(a_toked)

        # 답변 labels = [mask, mask, ...., mask, ..., <bos>,..답변.. <eos>, <pad>....]
        labels = [self.mask,] * q_len + a_toked[1:]

        # mask = 질문길이 0 + 답변길이 1 + 나머지 0
        mask = [0] * q_len + [1] * a_len + [0] * (self.max_len - q_len - a_len)
        # 답변 labels을 index 로 만든다.
        labels_ids = self.tokenizer.convert_tokens_to_ids(labels)
        # 최대길이만큼 PADDING
        while len(labels_ids) < self.max_len:
            labels_ids += [self.tokenizer.pad_token_id]

        # 질문 + 답변을 index 로 만든다.
        token_ids = self.tokenizer.convert_tokens_to_ids(q_toked + a_toked)
        # 최대길이만큼 PADDING
        while len(token_ids) < self.max_len:
            token_ids += [self.tokenizer.pad_token_id]

        #질문+답변, 마스크, 답변
        return (token_ids, np.array(mask), labels_ids)

In [12]:
def collate_batch(batch):
    data = [item[0] for item in batch]
    mask = [item[1] for item in batch]
    label = [item[2] for item in batch]
    return torch.LongTensor(data), torch.LongTensor(mask), torch.LongTensor(label)

In [13]:
torch.cuda.is_available()

True

In [14]:
#토큰/과/ 모델/의 /개념
#ㅌ/ㅗ/ㅋ/ㅡ_/ㄴ/
#0/1/2/3/4 -> 이건 토큰이구나!
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_set = ChatbotDataset(chatbot_data, max_len=40)
#윈도우 환경에서 num_workers 는 무조건 0으로 지정, 리눅스에서는 2
train_dataloader = DataLoader(train_set, batch_size=32, num_workers=0, shuffle=True, collate_fn=collate_batch,)


In [15]:
train_dataloader

In [16]:
learning_rate = 3e-5
criterion = torch.nn.CrossEntropyLoss(reduction="none")
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
epoch = 10
Sneg = 0.0000000001

In [17]:
model.to(device)
model.train()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=51200, bias=False)
)

In [18]:
from tqdm import tqdm
for epoch in range(epoch):
  t = tqdm(list(enumerate(train_dataloader)))
  for batch_idx, samples in t:
    optimizer.zero_grad()
    token_ids, mask, label = [i.to(device) for i in samples]
    out = model(token_ids)
    out = out.logits      # huggingface의 output에서 확인할 정보를 얻어냅니다.
    mask_3d = mask.unsqueeze(dim=2).repeat_interleave(repeats=out.shape[2], dim=2)
    mask_out = torch.where(mask_3d == 1, out, Sneg * torch.ones_like(out))
    loss = criterion(mask_out.transpose(2, 1), label)
    # 평균 loss 만들기
    avg_loss = loss.sum() / mask.sum()
    avg_loss.backward()
    # 학습 끝
    optimizer.step()
    t.set_postfix({"loss":avg_loss.item()})
print ("end")

<ipython-input-12-495d11317178>:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  return torch.LongTensor(data), torch.LongTensor(mask), torch.LongTensor(label)
100%|██████████| 370/370 [02:17<00:00,  2.70it/s, loss=27.2]

end


In [ ]:
model.to("cpu")
model.eval()
with torch.no_grad():
    while 1:
        q = input("user > ").strip()
        # 대화를 중단하기 위해서는 quit를 입력해주세요.
        if q == "quit":
            break
        a = ""
        while 1:
            input_ids = torch.LongTensor(koGPT2_Tokenizer.encode(Q_TKN + q + SENT + A_TKN + a)).unsqueeze(dim=0)
            pred = model(input_ids)
            pred = pred.logits
            gen = koGPT2_Tokenizer.convert_ids_to_tokens(torch.argmax(pred, dim=-1).squeeze().numpy().tolist())[-1]
            if gen == EOS:
                break
            a += gen.replace("▁", " ")
        print("Chatbot > {}".format(a.strip()))

user > 안녕
Chatbot > 안녕하세요
user > 반가워 너의 이름은 뭐야
Chatbot > 저는 위로해드리는 로봇이에요
user > 날 위로해줘
Chatbot > 정말 많이 힘들었죠
user > 진짜 너무 힘들어
Chatbot > 많이 힘들었죠
user > 다른 말로 위로 해줘
Chatbot > 정말 많이 힘들었죠  때론 힘내려 노력하지 않아도 돼요
user > 죽을 것 같이 힘든걸...
Chatbot > 지금은 괜찮길 바랄게요
user > 전혀 괜찮지 않아...
Chatbot > 완전히 괜찮아질 거예요
user > 그렇게 생각하고 버틴게 3년이야
Chatbot > 많은 시간이 흘렀네요
user > 그래도 나아지질 않아
Chatbot > 나아질 거예요
user > 구렁텅이로 빠지고 있어
Chatbot > 빠지고 싶은 마음이 굴뚝 같네요
